### Optimising a Single Elevator

The goal of the current exercise is to optimise the distance travelled by a single elevator in an online context. An elevator is defined as below (forgive me for becoming what I hate most, an object-oriented programmer): 

In [1]:
class Cart:
    # floor: int
    # direction: int; -1 -> down, 0 -> stationary, 1 -> up
    # distance: int; lifetime floor changes
    def __init__(self, floor = 0):
        self.floor = floor
        self.state = 0
        self.distance = 0
    
    # run every system tick to update floor
    def update_position(self):
        self.floor += self.state
        self.distance += abs(self.state)
    
    # change direction
    def set_state(self, dir): self.state = dir
    
    # floor getter
    def get_floor(self): return self.floor
    
    # state getter
    def get_state(self): return self.state
    
    # distance getter
    def get_distance(self): return self.distance

A request is defined as below: 

In [2]:
class Request:
    # c_floor: int; current floor (original request)
    # dir: int; -1 -> down, 1 -> up (original request)
    # t_floor: int; target floor (delayed request)
    def __init__(self, c_floor, dir, t_floor):
        self.c_floor = c_floor
        self.direction = dir
        self.t_floor = t_floor
        
        if not self.verify(): raise Exception("what")
    
    # checks if the request makes sense
    # mostly for sanity once I do fuzz testing
    def verify(self):
        if self.direction == -1: return self.t_floor < self.c_floor
        if self.direction == 1: return self.t_floor > self.c_floor
        return False
    
    # get initial request
    def get_initial_request(self):
        return self.c_floor, self.direction
    
    # reveal target floor only if the elevator is on
    # the current floor
    def get_target_floor(self, elevator):
        if elevator.get_floor() == self.c_floor: return self.t_floor
        else: return None
    
    def __lt__(self, other):
        return self.c_floor < other.c_floor
    
    def __eq__(self, other):
        if not isinstance(other, Request): return False
        return self.c_floor == other.c_floor
    
    def __str__(self):
        return f"Request floor: {self.c_floor}, Request direction: {self.direction}, Destination floor: {self.t_floor}"

Now, to create a way to simulate the elevator! 

In [3]:
from elevator import *
import heapq as pq

class Elevator_Simulator:
    def __init__(self, name, LO, HI, events = [], debug = False):
        self.LO, self.HI = LO, HI
        self.elevator = Cart()
        
        self.name = name
        self.debug = debug
        self.targets = set()
        self.next_targets = [] # prio queue for requests in opposite direction
        self.missed_dir_requests = [] # prio queue for requests in the same direction but not at an appropriate floor
        self.current_dir_requests = {} # hash map with request objects
        
        self.events = events
        self.event_idx = 0
    
    def add_event(self, event = None): self.events.append(event)

Done with some terrible code preamble—let's talk about the algorithm before I bore you with more Python. 

### What does an elevator need to do? 

Simply put, it needs to take passengers from floor x to floor y. A request is two-fold, one to call an elevator to the passenger's current floor (along with the direction they'd like to go in) and second, once the passenger is in the cart, they may reveal the floor that they want to go to. 

### Elevator not paternoster

I don't want the elevator to move when it doesn't need to, nor do I want it to change directions when it still has requests to handle in that direction. Another constraint is that it should not contain passengers when the elevator switches directions. 

### Solving it 

When an elevator is going in a direction, say UP, then each request can be classified into one of the three types: 
- Current direction request:
- Reverse direction request:
- Missed current direction request: 